Курсовой проект "Создание чат-бота в Telegram" по курсу "Введение в обработку естественного языка"
---

## Ноутбук № 3: рабочий код бота

Тот же самый код реализован в файле **spam_bot.py**

Устанавливаем библиотеку python-telegram-bot. Ссылка на документацию https://github.com/python-telegram-bot/python-telegram-bot

In [ ]:
#!pip install python-telegram-bot --upgrade

In [1]:
import pandas as pd
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import logging
import pickle
import os
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import linear_model

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

In [2]:
# путь к данным
path = '../../../data/spam_detection/'

## Готовим модель

In [3]:
# предобработка текста
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

In [4]:
# загружаем обработанный датафрейм
df = pd.read_pickle(os.path.join(path, 'df_processed.pkl'))
print(df.shape)
df.head(3)

(18393, 2)


,msg,spam
0,че за возня опять возле верный,0
1,что то похожий на стрельба быть и сейчас три м...,0
2,а ктоть в курс что за стрельба,0


In [5]:
# загружаем обученную модель
pkl_path_filename = os.path.join(path, 'lr_model.pkl')
with open(pkl_path_filename, 'rb') as file:
    lr = pickle.load(file)
lr

LogisticRegression(class_weight='balanced')

In [6]:
# загружаем CountVectorizer
pkl_path_filename = os.path.join(path, 'count_vectorizer.pkl')
with open(pkl_path_filename, 'rb') as file:
    count_vect = pickle.load(file)
count_vect

HashingVectorizer(n_features=4800)

In [7]:
# функция проверки сообщения на спам
def check_spam(text):
    # предоработка текста сообщения
    text = preprocess_text(text)
    # преобразование текста в Series, преобразование через CountVectorizer и предсказание
    return lr.predict(count_vect.transform(pd.Series(text)))

## Телеграм бот

In [8]:
# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO
                    )

logger = logging.getLogger()

In [9]:
# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Привет!')

def echo(update: Update, context: CallbackContext):
    msg = update.message.text
    # отвечаем на сообщение
    update.message.reply_text('Ваше сообщение: ' + msg)
    
    # проверяем сообщение на спам и отвечаем соотвтетсвующе
    if check_spam(msg) == 1:
        update.message.reply_text('Это спам!')
    else:
        update.message.reply_text('Это не спам')

In [10]:
# читаем токен телеграм-бота из файла и сохраняем в переменную
with open(os.path.join(path, 'token.txt'), "r") as file:
    token = file.read()

In [11]:
#Настройки 
# use_context (bool, optional) – If set to True uses the context based callback API (ignored 
# if dispatcher argument is used). Defaults to True. New users: set this to True.

updater = Updater(token=token, use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

In [12]:
# добавляем обработчики команд (handler) в диспетчере обработчиков

# обработчик команд - /start
dispatcher.add_handler(CommandHandler("start", start))

# обработчик сообщений
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

In [13]:
# Start the Bot - Начинаем поиск обновлений
updater.start_polling(drop_pending_updates=True)
# drop_pending_updates (bool, optional) – Whether to clean any pending updates on 
# Telegram servers before actually starting to poll. Default is False

# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()

2021-08-29 17:00:28,766 - apscheduler.scheduler - INFO - Scheduler started
2021-08-29 17:01:18,308 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-08-29 17:01:18,308 - apscheduler.scheduler - INFO - Scheduler has been shut down


telegram.ext подмодуль, предоставлющий простой в использовании интерфейс. Он состоит из нескольких классов, но двумя наиболее важными из них являются telegram.ext.Updater и telegram.ext.Dispatcher.
Класс Updater непрерывно получает новые обновления  из телеграмма и передает их в Dispatcher класс.
Если вы создадите Updater объект, он создаст Dispatcher для вас и свяжет их вместе.


Затем вы можете зарегистрировать обработчики разных типов в Dispatcher, который будет сортировать обновления, извлеченные в Updater соответствии с зарегистрированными вами обработчиками, и доставлять их в функцию обратного вызова, которую вы определили.


Напишем 2 обработчика команд. Это callback-функции, которые будут вызываться тогда, когда будет получено обновление. Напишем две таких функции для команды /start и для обычного любого текстового сообщения. В качестве аргументов туда передаются два параметра: bot и update. Bot содержит необходимые методы для взаимодействия с API, а update содержит данные о пришедшем сообщении.


Каждый обработчик является экземпляром любого подкласса класса telegram.ext.Handler . Библиотека предоставляет классы-обработчики почти для всех вариантов использования, но если вам нужно что-то очень конкретное, вы также можете создать подкласс Handler самостоятельно.

Теперь осталось лишь присвоить уведомлениям эти обработчики и начать поиск обновлений.
Цель состоит в том, чтобы вызывать эту функцию каждый раз, когда бот получает сообщение Telegram, содержащее /start команду. Для этого можно использовать CommandHandler(один из предоставленных подклассов Handler) и зарегистрировать его в диспетчере:
Класс Filters содержит ряд функций, которые фильтруют входящие сообщения на наличие текста, изображений, обновлений статуса и т.д. 
Любое сообщение, которое возвращает True в хотя бы одном из переданных фильтров, MessageHandler примет. Также можно написать свои собственные фильтры.


Создаём папку Bot, в которой потом создаём файл bot.py. Вы можете создать просто текстовый блокнот и вместо расширения .txt напишите .py Собираем код нашего бота. Открываем консоль и переходим в директорию с файлом, и запускаем python3 bot.py Бот будет работать пока будет открыто окно консоли.

In [6]:
# # Обработка команд
# def startCommand(bot, update):
#     bot.send_message(chat_id=update.message.chat_id, text='Добрый день')
    
# def textMessage(bot, update):
#     response = 'Ваше сообщение принял ' + update.message.text # формируем текст ответа
#     bot.send_message(chat_id=update.message.chat_id, text=response)

In [ ]:
# Добавляем хендлеры в диспетчер
# dispatcher.add_handler(start_command_handler)
# dispatcher.add_handler(text_message_handler)

In [ ]:
# # Хендлеры
# start_command_handler = CommandHandler('start', startCommand)
# text_message_handler = MessageHandler(Filters.text, textMessage)
# # Добавляем хендлеры в диспетчер
# dispatcher.add_handler(start_command_handler)
# dispatcher.add_handler(text_message_handler)
# # Начинаем поиск обновлений
# updater.start_polling(clean=True)
# # Останавливаем бота, если были нажаты Ctrl + C
# updater.idle()